In [1]:
import os
import numpy as np
from pathlib import Path
from bioio import BioImage
from glob import glob
import matplotlib.pyplot as plt
import bioio_czi
import imagej
from skimage import io
from skimage.filters import meijering
import segmentation as sg
from skimage.transform import resize
from skimage import util
from skimage import filters
import time
import shutil
from natsort import natsorted
import re
import os
import warnings
warnings.filterwarnings('ignore')
os.environ['JAVA_HOME']=r'C:\Users\zeiss\Downloads\fiji-win64\Fiji.app\java\win64\zulu8.60.0.21-ca-fx-jdk8.0.322-win_x64\jre\bin'

ModuleNotFoundError: No module named 'segmentation'

In [2]:
import imagej

# initialize ImageJ2 with Fiji plugins
ij = imagej.init('sc.fiji:fiji')
print(f"ImageJ2 version: {ij.getVersion()}")

ImageJ2 version: 2.15.0/1.54f


In [4]:
folders = Path('G:\Data\Zeiss\Chip_growth_experiments\growth_tracking').glob('*/')
folders = natsorted(list(folders))
folders 

[WindowsPath('G:/Data/Zeiss/Chip_growth_experiments/growth_tracking/20240529_C2_0.50mM_Myr_G40'),
 WindowsPath('G:/Data/Zeiss/Chip_growth_experiments/growth_tracking/20240816_C2_0.50mM_Myr_G41'),
 WindowsPath('G:/Data/Zeiss/Chip_growth_experiments/growth_tracking/20240821_A5_0.50mM_Myr_and_Ctrl_G42'),
 WindowsPath('G:/Data/Zeiss/Chip_growth_experiments/growth_tracking/20240823_C2_0.50mM_Myr,CtrL_and_100nM_G24_G43'),
 WindowsPath('G:/Data/Zeiss/Chip_growth_experiments/growth_tracking/20240829_A5_0.50mM_Myr_G24100nM,_+_root extract,_Ctrl_G44ABCD'),
 WindowsPath('G:/Data/Zeiss/Chip_growth_experiments/growth_tracking/20240830_C2_MSR_Myr,+G24100nM,+root_extract_G45ABCD'),
 WindowsPath('G:/Data/Zeiss/Chip_growth_experiments/growth_tracking/20240905_A5_0.50mM_Myr_G46A'),
 WindowsPath('G:/Data/Zeiss/Chip_growth_experiments/growth_tracking/20241108_C2_0.50mM_Myr_G48A-F'),
 WindowsPath('G:/Data/Zeiss/Chip_growth_experiments/growth_tracking/20241125_C2_0.50mM_Myr_G49ABC')]

In [14]:
pattern = re.compile(r'dapi', re.IGNORECASE)

for folder in folders:
   days =natsorted(list(folder.glob('*.czi')))
   # Find .czi files that match the pattern
   dapi_files = []
   for folder in folders:
       dapi_files.extend([f for f in folder.glob('*.czi') if pattern.search(f.name)])

In [17]:
# Get an AICSImage object
for dapi_file in dapi_files:
    print(dapi_file)
    img = BioImage(dapi_file, reconstruct_mosaic=False)
    dask_data = img.get_image_dask_data('SMZYX')
    print(dask_data.shape)

G:\Data\Zeiss\Chip_growth_experiments\growth_tracking\20240529_C2_0.50mM_Myr_G40\G40_A_C2_Fixed_DAPI_NileRed_restaining_after~5hour_gHCL_TritonX100_4%_wash.czi
(1, 80, 13, 2056, 2464)
G:\Data\Zeiss\Chip_growth_experiments\growth_tracking\20240529_C2_0.50mM_Myr_G40\G40_A_C2_Fixed_DAPI_NileRed_staining.czi
(1, 80, 14, 2056, 2464)
G:\Data\Zeiss\Chip_growth_experiments\growth_tracking\20240529_C2_0.50mM_Myr_G40\G40_A_C2_Fixed_DAPI_NileRed_staining_stitched.czi
(1, 80, 14, 2056, 2464)
G:\Data\Zeiss\Chip_growth_experiments\growth_tracking\20240529_C2_0.50mM_Myr_G40\G40_A_C2_Fixed_DAPI_NileRed_staining_stitched_projected.czi
(1, 80, 1, 2056, 2464)
G:\Data\Zeiss\Chip_growth_experiments\growth_tracking\20240529_C2_0.50mM_Myr_G40\G40_A_C2_Fixed_DAPI_staining.czi
(1, 80, 13, 2056, 2464)
G:\Data\Zeiss\Chip_growth_experiments\growth_tracking\20240529_C2_0.50mM_Myr_G40\G40_A_C2_Fixed_DAPI_staining_stitched.czi
(1, 80, 13, 2056, 2464)
G:\Data\Zeiss\Chip_growth_experiments\growth_tracking\20240529_C2_

In [31]:
for folder in folders:
    days =natsorted(list(folder.glob('*.czi')))
    for day in days:
        #create savefolder
        savefolder_string =  re.sub(r'_[dD]ay\d+', '', day.stem)
        savefolder = Path('processed_250326',savefolder_string)
        savefolder.mkdir(parents=True,exist_ok=True)
        #save processed files
        filename = f'{Path(day.stem)}.tif'
        if Path(savefolder,'Masks',filename).exists()!=True:
            try:
                #print current file
                print(filename)
                
                # Get an AICSImage object
                img = BioImage(day, reconstruct_mosaic=False)
                dask_data = img.get_image_dask_data('MZYX', S=2)
                print(dask_data.shape)
    
                #extract number of rows and cols from mosaic 
                xs,ys = [],[]
                positions = img.get_mosaic_tile_positions()
                for pos in positions:
                    xs.append(pos[0])
                    ys.append(pos[1])
                number_of_rows = len(np.unique(xs))
                number_of_cols = len(np.unique(ys))
                
                # Zproject tiles
                time.sleep(20)
                sg.Zproject_stack(dask_data)
                
                # Stitch image
                input_folder = 'G:/Data/Zeiss/Chip_growth_experiments/temp'
                output_folder = 'G:/Data/Zeiss/Chip_growth_experiments/temp'
                macro = f"""
                run("Grid/Collection stitching", "type=[Grid: snake by rows] order=[Right & Down                ] grid_size_x={number_of_cols} grid_size_y={number_of_rows}  tile_overlap=10 first_file_index_i=0 directory={input_folder} file_names=tile_{{i}}.tif output_textfile_name=TileConfiguration.txt fusion_method=[Linear Blending] regression_threshold=0.01 max/avg_displacement_threshold=1.50 absolute_displacement_threshold=2.50 compute_overlap subpixel_accuracy computation_parameters=[Save computation time (but use more RAM)] image_output=[Write to disk] output_directory={input_folder}");
                """
                result = ij.py.run_macro(macro)
                
                # resize stitched image
                path = fr'.\temp\img_t1_z1_c1'
                img = io.imread(path)
                resized_img = resize(img,np.array(img.shape)//(6,6))
                
                # enhance ridges
                ridge_img = sg.bowler_hat(util.invert(resized_img),no=20,si=15)
                
                # threshold ridges
                labels = sg.threshold_mask(ridge_img)
        
                #save file
                Zproject = Path(savefolder,'Zproject')
                Zproject.mkdir(parents=True,exist_ok=True)
                io.imsave(Path(Zproject, filename),img)
        
                Resized = Path(savefolder,'Resized')
                Resized.mkdir(parents=True,exist_ok=True)
                io.imsave(Path(Resized, filename),resized_img)
        
                Ridge = Path(savefolder,'Ridge')
                Ridge.mkdir(parents=True,exist_ok=True)
                io.imsave(Path(Ridge, filename),ridge_img)
        
                Masks = Path(savefolder,'Masks')
                Masks.mkdir(parents=True,exist_ok=True)
                io.imsave(Path(Masks, filename),labels)
                
                time.sleep(15)        
                shutil.rmtree('temp')
            except: 
                    print("couldn't process image") 
        else : 
            print('skipping already processed file')


skipping already processed file
skipping already processed file
skipping already processed file
skipping already processed file
skipping already processed file
skipping already processed file
skipping already processed file
skipping already processed file
skipping already processed file
skipping already processed file
skipping already processed file
skipping already processed file
skipping already processed file
skipping already processed file
skipping already processed file
skipping already processed file
skipping already processed file
skipping already processed file
skipping already processed file
skipping already processed file
skipping already processed file
skipping already processed file
skipping already processed file
skipping already processed file
skipping already processed file
skipping already processed file
skipping already processed file
skipping already processed file
skipping already processed file
skipping already processed file
skipping already processed file
skipping